In [1]:
import numpy as np
import torch
from torch import nn
import timm
from torchinfo import summary
from networks.model import DenseDepth

/home/iasl_server/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = DenseDepth(encoder_model="mobilevitv2")

ckpt = torch.load("/home/model_disk/DenseDepth/0504-1342/006-mobilevitv2-2.6988916897119903.pt", map_location=torch.device('cpu'))
model.load_state_dict(ckpt["model"])
model.eval()

summary(model, (1, 3, 480, 640), depth=4)

/home/iasl_server/miniconda3/envs/pytorch/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/iasl_server/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                                                 Output Shape              Param #
DenseDepth                                                             [1, 1, 480, 640]          --
├─Encoder: 1-1                                                         [1, 3, 480, 640]          --
│    └─ByobNet: 2-1                                                    --                        --
│    │    └─ConvNormAct: 3-1                                           [1, 64, 240, 320]         --
│    │    │    └─Conv2d: 4-1                                           [1, 64, 240, 320]         1,728
│    │    │    └─BatchNormAct2d: 4-2                                   [1, 64, 240, 320]         128
│    │    └─Sequential: 3-2                                            --                        --
│    │    │    └─Sequential: 4-3                                       [1, 128, 240, 320]        26,496
│    │    │    └─Sequential: 4-4                                       [1, 256, 120, 16

In [ ]:
features = []

def hook(module, input, output):
    features.append(output)

model.encoder.model.stem.register_forward_hook(hook)
model.encoder.model.stages

In [20]:
for i in model.encoder.model.stages.modules():
    print(i)
    print("-"*10)

Sequential(
  (0): Sequential(
    (0): BottleneckBlock(
      (conv1_1x1): ConvNormAct(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
      )
      (conv2_kxk): ConvNormAct(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
        (bn): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
      )
      (conv2b_kxk): Identity()
      (attn): Identity()
      (conv3_1x1): ConvNormAct(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
     